In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()  # Accelerate 기본환경 설정 (GPU 하나 사용, 혼합정밀도 등)

In [ ]:
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

# 디바이스 설정 (GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 재현성 위한 시드 고정
torch.manual_seed(42)

In [ ]:
import pandas as pd
from open_clip.tokenizer import SimpleTokenizer
# ---------------- 1) 데이터 전처리 --------------- #
def filter_captions(csv_in="./train.csv", csv_out="./train_filtered.csv", max_tok=75):
    tok = SimpleTokenizer()
    df = pd.read_csv(csv_in)
    ok = df["caption"].apply(lambda t: len(tok.encode(str(t))) <= max_tok)
    df[ok].reset_index(drop=True).to_csv(csv_out, index=False)
    print(f"[filter] {len(df)-ok.sum()} rows dropped → {csv_out}")

filter_captions()

In [ ]:
from torchvision import transforms

# CSV 파일 로드 (train.csv 경로는 상황에 맞게 지정)
train_df = pd.read_csv("train_filtered.csv")

# 이미지 전처리 Transform 정의
IMG_SIZE = 512
# 컬러 이미지: 리사이즈 -> 센터크롭 -> Tensor로 -> -1~1 정규화
img_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),  # 0~1로 변환
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # RGB 정규화 -> -1~1
])
# 흑백 입력 이미지: 리사이즈 -> 센터크롭 -> grayscale->Tensor로 (정규화는 생략하여 0~1 사용)
cond_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),
    transforms.Grayscale(num_output_channels=3),  # 3채널 그레이스케일:contentReference[oaicite:6]{index=6}
    transforms.ToTensor()  # 0~1로 변환 (정규화 없음)
])

class ColorizationDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, img_transforms, cond_transforms):
        self.df = dataframe
        self.img_transforms = img_transforms
        self.cond_transforms = cond_transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        # 파일 경로
        gray_path = row["input_img_path"]
        color_path = row["gt_img_path"]
        prompt = row["caption"]
        # 이미지 로드
        gray_img = Image.open(gray_path).convert("RGB")
        color_img = Image.open(color_path).convert("RGB")
        # 전처리
        cond_image = self.cond_transforms(gray_img)    # (3, H, W), 0~1
        target_image = self.img_transforms(color_img)  # (3, H, W), -1~1
        return cond_image, prompt, target_image

# Dataset 및 DataLoader 생성
train_dataset = ColorizationDataset(train_df, img_transforms, cond_transforms)
# 메모리 고려하여 batch_size 설정; A100 40GB 기준으로 8~16까지 가능. 여기서는 예시로 8.
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)


In [ ]:
from diffusers import (
    UNet2DConditionModel,
    AutoencoderKL,
    ControlNetModel,
    DDPMScheduler,
)
from transformers import CLIPTextModel, CLIPTokenizer
from peft import LoraConfig               # ← 여기서 가져옵니다!

device = "cuda"
pretrained_model = "stabilityai/stable-diffusion-2-1-base"

# --- 모델 로드 ---
vae          = AutoencoderKL.from_pretrained(pretrained_model, subfolder="vae").to(device)
text_encoder = CLIPTextModel.from_pretrained(pretrained_model, subfolder="text_encoder").to(device)
tokenizer    = CLIPTokenizer.from_pretrained(pretrained_model, subfolder="tokenizer")
unet         = UNet2DConditionModel.from_pretrained(pretrained_model, subfolder="unet")  # 아직 CPU

# --- LoRA 설정 (한 줄) ---
lora_cfg = LoraConfig(
    r=4,                    # 랭크
    lora_alpha=4,           # 스케일
    init_lora_weights="gaussian",
    target_modules=["to_k", "to_q", "to_v", "to_out.0"],  # 기본 어텐션 4개
)
unet.add_adapter(lora_cfg)   # LoRA 어댑터 삽입

unet.to(device)              # 그다음 GPU로 이동

print(
    "Trainable params:",
    sum(p.numel() for p in unet.parameters() if p.requires_grad),
)


In [ ]:
import torch.optim as optim

lora_params = filter(lambda p: p.requires_grad, unet.parameters())
optimizer = optim.AdamW(lora_params, lr=1e-4)

In [ ]:
from accelerate import Accelerator

# 가중치 dtype 설정 (fp16 훈련을 위해 half로)
weight_dtype = torch.float16
vae.requires_grad_(False)  # VAE는 학습하지 않음 (동결)
text_encoder.requires_grad_(False)  # 텍스트 인코더도 동결 (필요시 풀 수 있음)
unet.train()  # UNet (LoRA) 학습 모드

# Accelerator 설정 (fp16 혼합 정밀도)
accelerator = Accelerator(mixed_precision="fp16")
# 모델 및 옵티마이저, 데이터로더를 accelerator에 등록
unet, optimizer, train_loader = accelerator.prepare(unet, optimizer, train_loader)

num_epochs = 1  # 에폭 수 (필요에 따라 증가)
logging_steps = 100  # 로그 출력 간격
global_step = 0

In [ ]:
weight_dtype = torch.float16                # fp16
fp32         = torch.float32                # 가독성용

scheduler = DDPMScheduler.from_pretrained(
    pretrained_model, subfolder="scheduler"
)

In [ ]:
import time
from tqdm.auto import tqdm                 # Colab에 기본 포함

total_steps   = num_epochs * len(train_loader)   # 전체 step 수
global_step   = 0
progress_bar  = tqdm(total=total_steps, desc="Training", unit="step")

t_start = time.time()

for epoch in range(num_epochs):
    for cond_images, prompts, target_images in train_loader:
        # ────────────── 인코딩 ───────────────────
        with torch.no_grad():
            # VAE 는 fp32 그대로, 입력만 fp32
            latents = vae.encode(
                target_images.to(device, dtype=fp32)
            ).latent_dist.sample()

            latents = latents.to(dtype=weight_dtype) * 0.18215

            noise = torch.randn_like(latents)          # ⇒ fp16
            timesteps = torch.randint(
                0, scheduler.num_train_timesteps,
                (latents.size(0),), device=device
            ).long()

            noisy_latents = scheduler.add_noise(
                latents, noise, timesteps
            )

            text_ids = tokenizer(
                list(prompts),
                padding="max_length",
                max_length=tokenizer.model_max_length,
                return_tensors="pt",
            ).input_ids.to(device)

            text_embeds = text_encoder(text_ids).last_hidden_state
            text_embeds = text_embeds.to(dtype=weight_dtype)   # ★ fp16

        # ────────────── UNet forward ───────────────
        model_pred = unet(
            noisy_latents,
            timesteps,
            encoder_hidden_states=text_embeds,
        ).sample                                         # fp16

        # ────────────── 손실(float32) & 역전파 ───────
        loss = torch.nn.functional.mse_loss(
            model_pred.float(),                          # cast to fp32
            noise.float(),                               # cast to fp32
        )

        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()

        # ----- 진행 정보 업데이트 -----
        global_step += 1
        progress_bar.update(1)             # bar +1
        # 50 step마다 ETA 계산해서 bar 제목에 표시
        if global_step % 50 == 0:
            elapsed = time.time() - t_start
            step_per_sec = global_step / elapsed
            eta_seconds  = (total_steps - global_step) / step_per_sec
            progress_bar.set_postfix(
                loss=f"{loss.item():.4f}",
                eta=f"{eta_seconds/60:5.1f} min",
            )

progress_bar.close()

In [ ]:
# LoRA 학습이 완료된 unet (학습 당시 accelerator로 감싸졌을 수 있으니 unwrap)
unet_lora = accelerator.unwrap_model(unet).to("cpu")  # CPU로 잠시 이동 (메모리 확보 목적)

In [ ]:
unet_lora.save_attn_procs(f"lora_weights_epoch{epoch}")

In [ ]:
import os, shutil
save_dir = f"lora_weights_epoch{epoch}"

# ❹ 백업 경로 지정 (원하는 폴더로 바꿔도 됨)
drive_ckpt_dir = f"/content/drive/MyDrive/0730uhyun/{save_dir}"
os.makedirs(os.path.dirname(drive_ckpt_dir), exist_ok=True)

# ❺ 폴더 단위 복사 (dirs_exist_ok=True → 동일 이름 폴더 있으면 덮어씀)
shutil.copytree(save_dir, drive_ckpt_dir, dirs_exist_ok=True)
print(f"[✔] Google Drive 백업 완료: {drive_ckpt_dir}")

In [ ]:
# 이제 base UNet (with LoRA)와 ControlNet 모두 있는 상태
# base 모델은 추론 시 필요하지만, 훈련 동안 동결시킴 (LoRA 가중치 포함)
# LoRA 학습 완료된 UNet
unet_lora    = UNet2DConditionModel.from_pretrained(
    pretrained_model, subfolder="unet"
).to(device, dtype=torch.float16)
unet_lora.load_attn_procs("lora_weights_epoch0")   # ← LoRA weight 로드
unet_lora.requires_grad_(False)               # UNet+LoRA ∴ 동결
unet_lora.eval()

In [ ]:
fp16  = torch.float16
device = "cuda"
base_unet = UNet2DConditionModel.from_pretrained(
    pretrained_model,
    subfolder="unet",
    torch_dtype=fp16            # ★ LoRA 미적용 순수 UNet
).to(device)

# 1) ControlNet 생성 (LoRA 無)
controlnet = ControlNetModel.from_unet(base_unet).to(device)

In [ ]:
# ControlNet의 파라미터만 학습 가능하도록
controlnet.train()

In [ ]:
from diffusers import StableDiffusionControlNetPipeline

fp16_dtype    = torch.float16
fp32_dtype    = torch.float32
lr_cnet       = 5e-5
num_epochs    = 3
logging_steps = 100
sample_steps  = 1000   # 샘플 저장 간격



# ───────────────────────────────────────────────
# 1. 데이터로더 (ColorizationDataset 은 앞서 정의)
train_dataset = ColorizationDataset(train_df, img_transforms, cond_transforms)
train_loader  = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True)

# 2. 스케줄러·옵티마이저
noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model,"scheduler")
optimizer_cnet  = optim.AdamW(controlnet.parameters(), lr=lr_cnet)

In [ ]:
base_unet.requires_grad_(False)

In [ ]:
print(base_unet.conv_in.weight.dtype, base_unet.conv_in.weight.device)

In [ ]:
print(controlnet.conv_in.weight.dtype, controlnet.conv_in.weight.device)

In [ ]:
# 3. Accelerate 준비
accelerator = Accelerator(mixed_precision="fp16")
controlnet, optimizer_cnet, train_loader = accelerator.prepare(
    controlnet, optimizer_cnet, train_loader
)
base_unet = accelerator.prepare(base_unet)   # 동결이라도 device 관리 위해

In [ ]:
from tqdm.auto import tqdm
import time
from diffusers import DDIMScheduler
import shutil
total_steps  = num_epochs * len(train_loader)

progress_bar = tqdm(
    range(total_steps),
    desc="CNet",
    unit="step",
    disable=not accelerator.is_local_main_process,  # 멀티 GPU면 보조 프로세스 숨김
)
global_step = 0

# ───────────────────────────────────────────────
# 4. 학습 루프


t0 = time.time()

for epoch in range(num_epochs):
    for step, (cond_imgs, prompts, tgt_imgs) in enumerate(train_loader):
        # fp16 텐서 준비
        cond_imgs = cond_imgs.to(fp16_dtype)
        with torch.no_grad():
            tgt_latents = vae.encode(
                tgt_imgs.to(fp32_dtype)
            ).latent_dist.sample().to(fp16_dtype) * 0.18215
        noise      = torch.randn_like(tgt_latents)
        timesteps  = torch.randint(
            0, noise_scheduler.num_train_timesteps,
            (tgt_latents.size(0),),
            device=cond_imgs.device,
        ).long()
        noisy_lat  = noise_scheduler.add_noise(tgt_latents, noise, timesteps)

        # 텍스트 임베딩
        text_ids   = tokenizer(list(prompts), padding="max_length",
                       max_length=tokenizer.model_max_length,
                       return_tensors="pt").input_ids.to(cond_imgs.device)
        with torch.no_grad():
            text_emb = text_encoder(text_ids).last_hidden_state.to(fp16_dtype)

        # ─── ① ControlNet forward (특성 추출) ───
        down_res_samples, mid_res_sample = controlnet(
            noisy_lat,
            timesteps,
            encoder_hidden_states=text_emb,
            controlnet_cond=cond_imgs,
            return_dict=False,
        )

        # ─── ② UNet forward (ControlNet 잔차 추가) ───

        pred = base_unet(
            noisy_lat, timesteps,
            encoder_hidden_states=text_emb,
            down_block_additional_residuals=down_res_samples,
            mid_block_additional_residual=mid_res_sample,
        ).sample                        # fp16

        # ─── ③ Loss & backward ───
        loss = torch.nn.functional.mse_loss(pred.float(), noise.float())
        accelerator.backward(loss)
        optimizer_cnet.step(); optimizer_cnet.zero_grad()

        global_step += 1
        progress_bar.update(1)


        # # ── 샘플 저장 ─────────────────────────────
        # if accelerator.is_main_process and (global_step % sample_steps == 0):
        #     gray_pil = transforms.ToPILImage()(cond_imgs[0].cpu()).convert("RGB")

        #     unet_fp16      = base_unet.half().to(device)
        #     ctrl_fp16      = accelerator.unwrap_model(controlnet).half().to(device)

        #     pipe = StableDiffusionControlNetPipeline(
        #         vae           = vae,
        #         text_encoder  = text_encoder,
        #         tokenizer     = tokenizer,
        #         unet          = unet_fp16,
        #         controlnet    = ctrl_fp16,
        #         scheduler     = DDIMScheduler.from_config(noise_scheduler.config),
        #         safety_checker= None,
        #         feature_extractor = None,        # 또는 CLIPFeatureExtractor.from_pretrained(...)
        #     ).to(device, dtype=fp16_dtype)

        #     img = pipe(prompt=prompts[0], image=gray_pil,
        #                num_inference_steps=30, guidance_scale=7.5).images[0]
        #     os.makedirs("cnet_samples", exist_ok=True)
        #     img.save(f"cnet_samples/step_{global_step:06d}.png")
        #     del pipe; torch.cuda.empty_cache()

        # ── 로그 ────────────────────────────────
        if (global_step % logging_steps == 0) and accelerator.is_main_process:
            elapsed = time.time() - t0
            spd     = global_step / elapsed
            eta     = (total_steps - global_step) / spd / 60
            print(f"step {global_step}/{total_steps}  "
                  f"loss {loss.item():.4f}  eta {eta:5.1f} min")

    # ───── 에폭 종료 시 가중치 저장 (메인 프로세스만) ─────
    if accelerator.is_main_process:
        save_dir = f"controlnet_epoch{epoch+1}"
        accelerator.unwrap_model(controlnet).save_pretrained(save_dir)
        print(f"[✔] epoch {epoch+1} weights saved → {save_dir}")

        # (선택) Google Drive 백업
        drive_dir = f"/content/drive/MyDrive/0730uhyun/{save_dir}"
        shutil.copytree(save_dir, drive_dir, dirs_exist_ok=True)

    accelerator.wait_for_everyone()   # 다중 GPU 동기화
    progress_bar.set_description(f"CNet epoch {epoch+1}/{num_epochs}")


progress_bar.close()